In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import classification_report

In [39]:
class EEGConvNetMiniV2(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, input_size, **kwargs):
        super(EEGConvNetMiniV2, self).__init__()
        self.input_size = input_size
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(self.input_size, 32, cached=True, normalize=False) #16
        self.conv2 = GCNConv(32, 64, cached=True, normalize=False) #32
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        # Global add pooling
        mean_pool = global_add_pool(x, batch=batch)
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out
    

In [25]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
    corrected_data_list = list()
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
        
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd
        
def print_classification_report(y_pred, y_true):
    print(classification_report(y_pred, y_true))
    

## Moments Pearson

In [26]:
path = Path('graphs/moments_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [27]:
model = EEGConvNetMiniV2(input_size=6)

In [40]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV2(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [29]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=3.430972	 acc=0.628205
Test  : epoch=1	 loss=84.756177	 acc=0.375000

Train : epoch=2	 loss=3.848046	 acc=0.653846
Test  : epoch=2	 loss=205.992184	 acc=0.437500

Train : epoch=3	 loss=3.868034	 acc=0.653846
Test  : epoch=3	 loss=186.129371	 acc=0.437500

Train : epoch=4	 loss=3.930236	 acc=0.653846
Test  : epoch=4	 loss=233.002404	 acc=0.437500

Train : epoch=5	 loss=2.678225	 acc=0.525641
Test  : epoch=5	 loss=6.168532	 acc=0.500000

Train : epoch=6	 loss=0.732116	 acc=0.358974
Test  : epoch=6	 loss=1.496186	 acc=0.500000

Train : epoch=7	 loss=1.934822	 acc=0.653846
Test  : epoch=7	 loss=61.935240	 acc=0.500000

Train : epoch=8	 loss=3.294632	 acc=0.653846
Test  : epoch=8	 loss=197.114158	 acc=0.500000

Train : epoch=9	 loss=3.674262	 acc=0.653846
Test  : epoch=9	 loss=197.771834	 acc=0.500000

Train : epoch=10	 loss=3.594534	 acc=0.653846
Test  : epoch=10	 loss=205.907710	 acc=0.500000



In [30]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## Moments PLI

In [31]:
path = Path('graphs/moments_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [32]:
model = EEGConvNetMiniV2(input_size=6)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV2(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [33]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=2.769923	 acc=0.628205
Test  : epoch=1	 loss=3933.494510	 acc=0.500000

Train : epoch=2	 loss=3.115666	 acc=0.653846
Test  : epoch=2	 loss=3930.098148	 acc=0.500000

Train : epoch=3	 loss=3.093483	 acc=0.653846
Test  : epoch=3	 loss=2848.591862	 acc=0.500000

Train : epoch=4	 loss=3.137881	 acc=0.653846
Test  : epoch=4	 loss=3490.535718	 acc=0.500000

Train : epoch=5	 loss=3.058767	 acc=0.653846
Test  : epoch=5	 loss=3538.052487	 acc=0.500000

Train : epoch=6	 loss=2.848548	 acc=0.653846
Test  : epoch=6	 loss=5908.999238	 acc=0.500000

Train : epoch=7	 loss=3.605641	 acc=0.653846
Test  : epoch=7	 loss=3769.113274	 acc=0.500000

Train : epoch=8	 loss=3.169668	 acc=0.653846
Test  : epoch=8	 loss=3931.694862	 acc=0.500000

Train : epoch=9	 loss=3.090617	 acc=0.653846
Test  : epoch=9	 loss=4048.143135	 acc=0.500000

Train : epoch=10	 loss=3.164154	 acc=0.653846
Test  : epoch=10	 loss=3919.030052	 acc=0.500000



In [34]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## Raw Pearson

In [35]:
path = Path('graphs/raw_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [41]:
model = EEGConvNetMiniV2(input_size=153550)

In [42]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4918578


EEGConvNetMiniV2(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [43]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=4.244460	 acc=0.512821
Test  : epoch=1	 loss=1.143702	 acc=0.562500

Train : epoch=2	 loss=3.826289	 acc=0.653846
Test  : epoch=2	 loss=19.339743	 acc=0.500000

Train : epoch=3	 loss=3.726480	 acc=0.653846
Test  : epoch=3	 loss=35.857325	 acc=0.500000

Train : epoch=4	 loss=3.783404	 acc=0.653846
Test  : epoch=4	 loss=40.925526	 acc=0.500000

Train : epoch=5	 loss=3.597779	 acc=0.653846
Test  : epoch=5	 loss=39.063034	 acc=0.500000

Train : epoch=6	 loss=3.656141	 acc=0.653846
Test  : epoch=6	 loss=41.830534	 acc=0.500000

Train : epoch=7	 loss=3.631634	 acc=0.653846
Test  : epoch=7	 loss=38.785208	 acc=0.500000

Train : epoch=8	 loss=3.641561	 acc=0.653846
Test  : epoch=8	 loss=39.220567	 acc=0.500000

Train : epoch=9	 loss=3.580200	 acc=0.653846
Test  : epoch=9	 loss=43.671839	 acc=0.500000

Train : epoch=10	 loss=3.556723	 acc=0.653846
Test  : epoch=10	 loss=42.050948	 acc=0.500000



In [44]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

### Raw PLI

In [45]:
path = Path('graphs/raw_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [46]:
model = EEGConvNetMiniV2(input_size = 153550)

In [47]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4918578


EEGConvNetMiniV2(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [48]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
print_classification_report(y_pred, y_true)

Train : epoch=1	 loss=2.345566	 acc=0.602564
Test  : epoch=1	 loss=200.664794	 acc=0.437500

Train : epoch=2	 loss=3.103518	 acc=0.653846
Test  : epoch=2	 loss=336.783863	 acc=0.500000

Train : epoch=3	 loss=3.084777	 acc=0.653846
Test  : epoch=3	 loss=526.255896	 acc=0.500000

Train : epoch=4	 loss=3.257207	 acc=0.653846
Test  : epoch=4	 loss=398.767859	 acc=0.437500

Train : epoch=5	 loss=3.146494	 acc=0.653846
Test  : epoch=5	 loss=366.304906	 acc=0.437500

Train : epoch=6	 loss=3.246460	 acc=0.653846
Test  : epoch=6	 loss=728.672698	 acc=0.500000

Train : epoch=7	 loss=3.234432	 acc=0.653846
Test  : epoch=7	 loss=744.302957	 acc=0.437500

Train : epoch=8	 loss=3.210530	 acc=0.653846
Test  : epoch=8	 loss=709.090102	 acc=0.437500

Train : epoch=9	 loss=3.231962	 acc=0.653846
Test  : epoch=9	 loss=736.884372	 acc=0.437500

Train : epoch=10	 loss=3.321220	 acc=0.653846
Test  : epoch=10	 loss=918.301716	 acc=0.500000

              precision    recall  f1-score   support

           0 

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## PSD CSD

In [49]:
path = Path('graphs/psd_csd/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [50]:
model = EEGConvNetMiniV2(input_size = 6)

In [51]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV2(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [52]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.743609	 acc=0.653846
Test  : epoch=1	 loss=0.692493	 acc=0.500000

Train : epoch=2	 loss=0.751658	 acc=0.653846
Test  : epoch=2	 loss=0.691064	 acc=0.500000

Train : epoch=3	 loss=0.687184	 acc=0.653846
Test  : epoch=3	 loss=0.693035	 acc=0.500000

Train : epoch=4	 loss=1.272649	 acc=0.653846
Test  : epoch=4	 loss=0.724387	 acc=0.500000

Train : epoch=5	 loss=2.070832	 acc=0.653846
Test  : epoch=5	 loss=0.889823	 acc=0.500000

Train : epoch=6	 loss=3.609978	 acc=0.653846
Test  : epoch=6	 loss=1.310445	 acc=0.500000

Train : epoch=7	 loss=3.948777	 acc=0.653846
Test  : epoch=7	 loss=1.986002	 acc=0.500000

Train : epoch=8	 loss=4.108884	 acc=0.653846
Test  : epoch=8	 loss=3.043207	 acc=0.500000

Train : epoch=9	 loss=3.806369	 acc=0.653846
Test  : epoch=9	 loss=3.595432	 acc=0.500000

Train : epoch=10	 loss=2.799958	 acc=0.653846
Test  : epoch=10	 loss=3.421238	 acc=0.500000



In [53]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## PSD Pearson

In [54]:
path = Path('graphs/psd_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_38120\960641861.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [55]:
model = EEGConvNetMiniV2(input_size=6)

In [56]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV2(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [57]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=2.837113	 acc=0.564103
Test  : epoch=1	 loss=6.143428	 acc=0.500000

Train : epoch=2	 loss=3.710146	 acc=0.653846
Test  : epoch=2	 loss=30.761767	 acc=0.500000

Train : epoch=3	 loss=3.719857	 acc=0.653846
Test  : epoch=3	 loss=37.149661	 acc=0.500000

Train : epoch=4	 loss=3.626029	 acc=0.653846
Test  : epoch=4	 loss=32.957040	 acc=0.500000

Train : epoch=5	 loss=3.571863	 acc=0.653846
Test  : epoch=5	 loss=37.050548	 acc=0.500000

Train : epoch=6	 loss=3.596621	 acc=0.653846
Test  : epoch=6	 loss=33.532514	 acc=0.500000

Train : epoch=7	 loss=3.555849	 acc=0.653846
Test  : epoch=7	 loss=29.540348	 acc=0.500000

Train : epoch=8	 loss=3.548315	 acc=0.653846
Test  : epoch=8	 loss=32.548591	 acc=0.500000

Train : epoch=9	 loss=3.531329	 acc=0.653846
Test  : epoch=9	 loss=32.677384	 acc=0.500000

Train : epoch=10	 loss=3.508580	 acc=0.653846
Test  : epoch=10	 loss=35.081911	 acc=0.500000



In [58]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le